In [162]:
import os
from dotenv import load_dotenv
from sqlalchemy import make_url
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
import pandas as pd
import uuid
from typing import Dict, List
from sqlalchemy import make_url
from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from tqdm import tqdm
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI
from tavily import TavilyClient
import json
from IPython.display import Markdown


### Load Environment

In [33]:
# load environment variables
load_dotenv()

# may need to change this to a different name
openai_api_key = os.getenv("OPENAI_API_KEY")
connection_string = os.getenv("DB_CONNECTION") 
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
url = make_url(connection_string)

### Connect to the database

In [3]:
# Create embedding model
embedding_model = OpenAIEmbedding(model="text-embedding-3-large")

# Set up database connection and vector store
url = make_url(connection_string)
vector_store = PGVectorStore.from_params(
    database='ai_advising_db',
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="alumni_records",
    embed_dim=3072,  # openai embedding dimension
    hybrid_search=True,  # Enable hybrid search
    text_search_config="english",
)

In [4]:
# Create a hybrid_index from an existing vector store and embedding model
hybrid_index = VectorStoreIndex.from_vector_store(vector_store=vector_store,
                                                  embed_model=embedding_model)

In [5]:
from prompts.prompt_template import (
    student_info_summary_template,
    query_diversification_template
)


### Simulate student information

In [6]:
# Section 1: Academic & Career Direction
q1 = "What are the student's primary academic interests and strengths?"
a1 = "Strong interest in AI and robotics, particularly excels at mathematical modeling and programming. Shows natural aptitude for problem-solving and algorithm development."

q2 = "What specific career paths or industries interest the student?"
a2 = "Interested in working as an AI researcher or robotics engineer in tech companies. Also curious about autonomous vehicle development."

q3 = "What are their favorite and least favorite courses so far?"
a3 = "Favorite: Machine Learning and Advanced Mathematics. Least favorite: Required general education writing courses."

# Section 2: Experience & Skills
q4 = "What relevant experience does the student have?"
a4 = "Led a university robotics club project, completed a summer internship at a local tech startup working on computer vision algorithms."

q5 = "What unique skills or achievements would they like to highlight?"
a5 = "Proficient in Python and TensorFlow, won 2nd place in regional AI competition, developed a small autonomous drone project."

# Section 3: Personal Factors
q6 = "What are their key extracurricular activities or passion projects?"
a6 = "Active member of robotics club, mentors high school students in programming, builds custom drones as a hobby."

q7 = "What factors are most important in their academic/career decisions?"
a7 = "Values innovation and cutting-edge technology, wants to work on projects with real-world impact, prefers collaborative environments."

# Section 4: Advisor Context
q8 = "Any additional observations or context about the student's goals/challenges?"
a8 = "Student shows exceptional potential in technical areas but could benefit from more interdisciplinary exposure. Sometimes hesitant to consider paths outside of pure technical roles."

In [7]:
# Dictionary structure for collecting form data
student_info = {
    "academic_interests": a1, # from q1
    "career_paths": a2,      # from q2
    "course_preferences": a3, # from q3
    "experience": a4,        # from q4
    "skills": a5,           # from q5
    "extracurriculars": a6, # from q6
    "decision_factors": a7, # from q7
    "advisor_notes": a8     # from q8
}

In [8]:
def format_context(student_info):
    """Format student info into context string for the template"""
    context_parts = []
    
    # Only include non-empty responses
    for question, answer in student_info.items():
        if answer and answer.strip():
            # Convert snake_case to readable format
            readable_question = question.replace('_', ' ').title()
            context_parts.append(f"{readable_question}:\n{answer}\n")
    
    return "\n".join(context_parts)


context = format_context(student_info)
summary_prompt = student_info_summary_template.format(context=context)
print(summary_prompt)


You are an AI academic advisor assistant tasked with analyzing student information to create an organized summary that will guide recommendation queries. Generate a structured summary using only the following sections, with no additional introduction or conclusion.
------------------------------------------------------------------------------------------------
Given student information:
Academic Interests:
Strong interest in AI and robotics, particularly excels at mathematical modeling and programming. Shows natural aptitude for problem-solving and algorithm development.

Career Paths:
Interested in working as an AI researcher or robotics engineer in tech companies. Also curious about autonomous vehicle development.

Course Preferences:
Favorite: Machine Learning and Advanced Mathematics. Least favorite: Required general education writing courses.

Experience:
Led a university robotics club project, completed a summer internship at a local tech startup working on computer vision algor

In [9]:
llm = OpenAI(model="gpt-4o")

## student_info_summary

In [10]:
llm_response = llm.complete(
            student_info_summary_template.format(context=context)
        )
        
llm_response_text = llm_response.text.strip()
print(llm_response_text)

1. CORE INTERESTS AND ABILITIES
- The student has a strong interest in AI and robotics, with a particular aptitude for mathematical modeling, programming, problem-solving, and algorithm development. These interests are supported by their favorite courses, Machine Learning and Advanced Mathematics, and their extracurricular activities, such as leading a robotics club project.
- A clear theme is the student's focus on technical and innovative fields, particularly in AI and robotics, which aligns with their career aspirations to become an AI researcher or robotics engineer.
- There is a strong alignment between the student's interests and experiences, as demonstrated by their internship in computer vision algorithms and their autonomous drone project.

2. EXPERIENCE AND SKILLS CONTEXT
- The student's experiences and skills cluster around AI, robotics, and programming. They have led a university robotics club project, completed an internship focused on computer vision, and developed an aut

## query_diversification

In [11]:
llm_response2 = llm.complete(
            query_diversification_template.format(summary=llm_response_text)
        )
        
llm_response_text2 = llm_response2.text.strip()
print(llm_response_text2)

Profiles of alumni who are AI researchers or robotics engineers with expertise in mathematical modeling and programming. 

Alumni working in fields related to AI and robotics, such as computer vision or autonomous systems, with skills in Python and TensorFlow.

Alumni who have led innovative technology projects with real-world impact in collaborative environments.

Alumni who have applied AI and robotics skills in interdisciplinary roles, such as healthcare technology or environmental solutions.


In [14]:
query_texts = [q.strip() for q in llm_response_text2.split('\n') if q.strip()]
for query in query_texts:
    print(query)


Profiles of alumni who are AI researchers or robotics engineers with expertise in mathematical modeling and programming.
Alumni working in fields related to AI and robotics, such as computer vision or autonomous systems, with skills in Python and TensorFlow.
Alumni who have led innovative technology projects with real-world impact in collaborative environments.
Alumni who have applied AI and robotics skills in interdisciplinary roles, such as healthcare technology or environmental solutions.


In [16]:
from typing import Dict, List, Tuple
from llama_index.core.schema import NodeWithScore
import asyncio
from llama_index.core.workflow import (
    Workflow,
    step,
    Context,
    Event,
)
from llama_index.core.workflow.events import (
    StartEvent,
    StopEvent,
)

class DatabaseSearchResultsEvent(Event):
    """Event containing database search results."""
    fusion_results_dict: Dict[str, List[NodeWithScore]]

In [132]:
class SearchWorkflow(Workflow):
    async def run_queries(self, queries: List[str], retrievers: List) -> Dict[Tuple[str, int], List[NodeWithScore]]:
        async def _retrieve(query: str, retriever, idx: int):
            result = await retriever.aretrieve(query)
            return (query, idx), result

        tasks = [
            _retrieve(query, retriever, idx)
            for query in queries
            for idx, retriever in enumerate(retrievers)
        ]

        results = await asyncio.gather(*tasks)
        return dict(results)

    async def fuse_results_per_query(
        self, 
        results_dict: Dict[Tuple[str, int], List[NodeWithScore]], 
        similarity_top_k: int = 5
    ) -> Dict[str, List[NodeWithScore]]:
        k = 60.0
        query_fusion_results = {}

        # Group results by query
        query_groups = {}
        for (query, _), nodes_with_scores in results_dict.items():
            if query not in query_groups:
                query_groups[query] = []
            query_groups[query].extend(nodes_with_scores)

        # Fuse results for each query
        for query, nodes_with_scores in query_groups.items():
            fused_scores = {}
            text_to_node = {}

            # Calculate reciprocal rank fusion scores
            for rank, node_with_score in enumerate(
                sorted(nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True)
            ):
                text = node_with_score.node.get_content()
                text_to_node[text] = node_with_score
                if text not in fused_scores:
                    fused_scores[text] = 0.0
                fused_scores[text] += 1.0 / (rank + k)

            # Sort by fusion scores
            reranked_results = dict(
                sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
            )

            # Create final reranked list
            reranked_nodes = []
            for text, score in reranked_results.items():
                reranked_nodes.append(text_to_node[text])
                reranked_nodes[-1].score = score

            query_fusion_results[query] = reranked_nodes[:similarity_top_k]

        return query_fusion_results

    @step
    async def perform_searches(self, ctx: Context, ev: StartEvent) -> StopEvent:

        queries = ev.get("search_query")

        # Setup retrievers with fixed top_k
        vector_retriever = hybrid_index.as_retriever(
            vector_store_query_mode="default",
            similarity_top_k=5
        )
        text_retriever = hybrid_index.as_retriever(
            vector_store_query_mode="sparse",
            similarity_top_k=5
        )

        # Run searches
        raw_results_dict = await self.run_queries(queries, [vector_retriever, text_retriever])
        
        # Fuse results
        fusion_results_dict = await self.fuse_results_per_query(
            raw_results_dict, 
            similarity_top_k=5
        )
        # format the results
        formatted_results = {query: [node.node.get_content() for node in nodes] for query, nodes in fusion_results_dict.items()}

        return StopEvent(formatted_results)

In [133]:
w_handler = SearchWorkflow(timeout=120, verbose=True).run(search_query = query_texts)
w_result = await w_handler

Running step perform_searches
Step perform_searches produced event StopEvent


In [135]:
for query, nodes in w_result.items():
    print(f'Query: {query}\n')
    for node in nodes:
        print(f'{node}')
    print('\n')



Query: Profiles of alumni who are AI researchers or robotics engineers with expertise in mathematical modeling and programming.

Major: Mathmatics; Degree: M.S; Job Title: Staff Machine Learning Engineer; Industry: Motor Vehicle Manufacturing
Major: Mathmatics; Degree: M.S; Job Title: Site Reliability Engineer; Industry: Computers and Electronics Manufacturing
Major: Computational Mathmatics; Degree: M.S; Job Title: Data Scientist; Industry: Software Development
Major: Mathmatics; Degree: PhD; Job Title: Senior System Engineer; Industry: Defense and Space Manufacturing
Major: Mathmatics; Degree: PhD; Job Title: Staff Data Engineer ; Industry: Software Development


Query: Alumni working in fields related to AI and robotics, such as computer vision or autonomous systems, with skills in Python and TensorFlow.

Major: Mathmatics; Degree: M.S; Job Title: Staff Machine Learning Engineer; Industry: Motor Vehicle Manufacturing
Major: Computational Mathmatics; Degree: M.S; Job Title: Data Scie

In [120]:
from llama_index.core.prompts.base import PromptTemplate

internet_search_template = PromptTemplate("""
You are a search query generation agent for an academic advising system. Your task is to generate two comprehensive search queries for finding enriching external information that complements alumni recommendations.

------------------------------------------------------------------------------------------------
Student Profile Summary:
{context}                                
------------------------------------------------------------------------------------------------

Generate 2 different types of search queries. Each query should be a detailed natural language phrase that integrates multiple aspects of the student's profile:

1. INDUSTRY TRENDS AND OPPORTUNITIES
Generate a comprehensive search query that combines:
- The student's primary field and interests
- Related or broader career paths
- Current industry developments
- Emerging opportunities and requirements
Format: Create a natural language search phrase that would surface articles, reports, and analyses about career trends and opportunities.

2. INSPIRATIONAL CAREER PATHS
Generate a search query focused on finding:
- Notable professionals in the student's field of interest
- Leaders who combined similar skills in innovative ways
- Success stories from related or adjacent fields
- Career progression examples that match student's values
Format: Create a natural language search phrase that would surface profiles, interviews, and career journey stories with actual names and details.

Important:
- Make queries detailed enough to find specific, relevant content
- Include key aspects from the student profile
- Focus on current and forward-looking information
- Ensure queries capture both traditional and innovative career perspectives
- Maintain natural language suitable for internet search engines

Return just the two queries without numbering or additional text.
""")

internet_search_prompt = internet_search_template.format(context=llm_response_text)
print(internet_search_prompt)



You are a search query generation agent for an academic advising system. Your task is to generate two comprehensive search queries for finding enriching external information that complements alumni recommendations.

------------------------------------------------------------------------------------------------
Student Profile Summary:
1. CORE INTERESTS AND ABILITIES
- The student has a strong interest in AI and robotics, with a particular aptitude for mathematical modeling, programming, problem-solving, and algorithm development. These interests are supported by their favorite courses, Machine Learning and Advanced Mathematics, and their extracurricular activities, such as leading a robotics club project.
- A clear theme is the student's focus on technical and innovative fields, particularly in AI and robotics, which aligns with their career aspirations to become an AI researcher or robotics engineer.
- There is a strong alignment between the student's interests and experiences, as d

In [121]:
llm_response3 = llm.complete(
            prompt=internet_search_prompt
        )
        
llm_response_text3 = llm_response3.text.strip()
print(llm_response_text3)

"What are the latest industry trends and emerging opportunities in AI and robotics, particularly for careers in AI research and robotics engineering, and how are these fields evolving with advancements in mathematical modeling, programming, and algorithm development?"

"Who are the notable professionals and leaders in AI and robotics who have successfully integrated skills in Python, TensorFlow, and computer vision, and what are their career progression stories that emphasize innovation, technical excellence, and real-world impact?"


In [122]:
llm_response3

CompletionResponse(text='"What are the latest industry trends and emerging opportunities in AI and robotics, particularly for careers in AI research and robotics engineering, and how are these fields evolving with advancements in mathematical modeling, programming, and algorithm development?"\n\n"Who are the notable professionals and leaders in AI and robotics who have successfully integrated skills in Python, TensorFlow, and computer vision, and what are their career progression stories that emphasize innovation, technical excellence, and real-world impact?"', additional_kwargs={}, raw=ChatCompletion(id='chatcmpl-Ac2YbQkaBRFdfjqLShRxHwlJ5kj7G', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='"What are the latest industry trends and emerging opportunities in AI and robotics, particularly for careers in AI research and robotics engineering, and how are these fields evolving with advancements in mathematical modeling, programming, and 

In [123]:
llm_response_text3_list = [part.strip('"') for part in llm_response3.text.split('\n\n')]
for query in llm_response_text3_list:
    print(query)


What are the latest industry trends and emerging opportunities in AI and robotics, particularly for careers in AI research and robotics engineering, and how are these fields evolving with advancements in mathematical modeling, programming, and algorithm development?
Who are the notable professionals and leaders in AI and robotics who have successfully integrated skills in Python, TensorFlow, and computer vision, and what are their career progression stories that emphasize innovation, technical excellence, and real-world impact?


In [124]:
class QuestionsGeneratedEvent(Event):
    """Event containing generated internet context search questions."""
    internet_search_questions: List[str]

In [125]:
class TopicResearchWorkflow(Workflow):
    def __init__(self, timeout: int = 60, verbose: bool = True, tavily: TavilyClient = None):
        super().__init__(timeout=timeout, verbose=verbose)
        self.tavily = tavily
        if not self.tavily:
            raise ValueError("Tavily client must be provided")

    @step
    async def generate_questions(self, ctx: Context, ev: StartEvent) -> QuestionsGeneratedEvent:
        llm_response_text = ev.get("context")
        if not llm_response_text:
            raise ValueError("No llm_response_text provided.")

        # Generate questions using the LLM
        result = await llm.acomplete(prompt=internet_search_template.format(context=llm_response_text))
        # Split into a clean two-item list
        result_list = [part.strip('"') for part in result.text.strip().split('\n\n')]

        return QuestionsGeneratedEvent(internet_search_questions=result_list)

    @step
    async def perform_internet_searches(self, ctx: Context, ev: QuestionsGeneratedEvent) -> StopEvent:
        search_questions = ev.internet_search_questions
        print("Internet Search Questions:")
        for i, question in enumerate(search_questions, 1):
            print(f"{i}. {question}")

        async def search_question(question, delay_seconds: float = 0.5):
            try:
                await asyncio.sleep(delay_seconds)
                context = await asyncio.get_event_loop().run_in_executor(
                    None,
                    lambda: self.tavily.qna_search(
                        query=question,
                        search_depth="advanced",
                        topic="general",
                        days=365,
                        max_results=6
                    )
                )
                return {"question": question, "answer": context} if context else None
            except Exception as e:
                print(f"Failed to perform Tavily QnA search for query: {question}")
                print(f"Error: {str(e)}")
                return None

        all_results = [
            result for result in
            await asyncio.gather(*[
                search_question(q, delay_seconds=i * 0.5)
                for i, q in enumerate(search_questions)
            ])
            if result is not None
        ]

        # Convert the list of results into a dictionary for easier handling:
        results_dict = {res['question']: res['answer'] for res in all_results}

        print("Context Search Results:")
        for q, a in results_dict.items():
            print(f"Question: {q}\nAnswer: {a}\n")

        # Return a StopEvent with a dictionary result
        return StopEvent(results_dict)



In [126]:
w_handler2 = TopicResearchWorkflow(timeout=60, verbose=True, tavily=tavily).run(context=llm_response_text)
w_result2 = await w_handler2

Running step generate_questions
Step generate_questions produced event QuestionsGeneratedEvent
Running step perform_internet_searches
Internet Search Questions:
1. What are the latest industry trends and emerging opportunities in AI and robotics, particularly for careers in AI research and robotics engineering, and how are skills in mathematical modeling, programming, and algorithm development shaping these fields?
2. Who are the notable AI researchers and robotics engineers who have successfully integrated skills in Python, TensorFlow, and computer vision, and what are their career journeys and success stories that align with values of innovation and real-world impact?
Context Search Results:
Question: What are the latest industry trends and emerging opportunities in AI and robotics, particularly for careers in AI research and robotics engineering, and how are skills in mathematical modeling, programming, and algorithm development shaping these fields?
Answer: In 2024, key trends in A

In [127]:
w_result2

{'What are the latest industry trends and emerging opportunities in AI and robotics, particularly for careers in AI research and robotics engineering, and how are skills in mathematical modeling, programming, and algorithm development shaping these fields?': 'In 2024, key trends in AI and robotics include the integration of AI in cybersecurity, where it is used to combat complex cyber threats, and the rise of mobile manipulators, combining collaborative robot arms and mobile robots for tasks in industries like automotive and logistics. The demand for service robots is also increasing significantly due to staff shortages, with sales rising by 48%.\n\nEmerging opportunities for careers in AI research and robotics engineering are shaped by advancements in areas such as machine learning, humanoid robots, and the use of digital twins that merge digital and physical realms for intelligent automation solutions. \n\nSkills in mathematical modeling, programming, and algorithm development are cr

In [129]:
for q, a in w_result2.items():
    display(Markdown(f"#### Question: {q}\n"))
    display(Markdown(f"{a}\n"))




#### Question: What are the latest industry trends and emerging opportunities in AI and robotics, particularly for careers in AI research and robotics engineering, and how are skills in mathematical modeling, programming, and algorithm development shaping these fields?


In 2024, key trends in AI and robotics include the integration of AI in cybersecurity, where it is used to combat complex cyber threats, and the rise of mobile manipulators, combining collaborative robot arms and mobile robots for tasks in industries like automotive and logistics. The demand for service robots is also increasing significantly due to staff shortages, with sales rising by 48%.

Emerging opportunities for careers in AI research and robotics engineering are shaped by advancements in areas such as machine learning, humanoid robots, and the use of digital twins that merge digital and physical realms for intelligent automation solutions. 

Skills in mathematical modeling, programming, and algorithm development are crucial as they underpin the design and functionality of AI systems and robotic applications. These competencies facilitate the development of sophisticated algorithms that drive machine learning models, enhance robotic capabilities, and optimize automation processes across various sectors.


#### Question: Who are the notable AI researchers and robotics engineers who have successfully integrated skills in Python, TensorFlow, and computer vision, and what are their career journeys and success stories that align with values of innovation and real-world impact?


Notable AI researchers and robotics engineers who have successfully integrated skills in Python, TensorFlow, and computer vision include:

1. **John J. Leonard**: An American roboticist and professor at MIT, Leonard is renowned for his research in simultaneous location and mapping (SLAM) for autonomous mobile robots. He led MIT's team in the 2007 DARPA Urban Challenge, focusing on innovative robotics solutions with real-world applications.

2. **Francois Chollet**: Creator of Keras and a significant figure in AI, Chollet emphasizes the potential of artificial general intelligence (AGI) as a means to advance human knowledge. His work has been influential in deep learning and LLMs, aligning with innovation and impactful technology.

3. **Ruslan Salakhutdinov**: An associate professor at Carnegie Mellon University and former director of AI research at Apple, Salakhutdinov has contributed to significant advancements in machine learning and AI, including leading efforts in general AI at Google and AlphaGo's development, which marked a milestone in AI's capabilities.

These individuals exemplify the integration of cutting-edge technology and research, driving innovation and addressing real-world challenges through their expertise in AI and robotics.


In [130]:
def structure_search_results(w_result, w_result2):
    combined_responses = {
        "alumni_profiles": w_result,  # Dictionary of alumni search results
        "internet_insights": w_result2  # Dictionary of internet-based insights
    }
    return combined_responses

In [139]:
combined_responses = structure_search_results(w_result, w_result2)
combined_responses


{'alumni_profiles': {'Profiles of alumni who are AI researchers or robotics engineers with expertise in mathematical modeling and programming.': ['Major: Mathmatics; Degree: M.S; Job Title: Staff Machine Learning Engineer; Industry: Motor Vehicle Manufacturing',
   'Major: Mathmatics; Degree: M.S; Job Title: Site Reliability Engineer; Industry: Computers and Electronics Manufacturing',
   'Major: Computational Mathmatics; Degree: M.S; Job Title: Data Scientist; Industry: Software Development',
   'Major: Mathmatics; Degree: PhD; Job Title: Senior System Engineer; Industry: Defense and Space Manufacturing',
   'Major: Mathmatics; Degree: PhD; Job Title: Staff Data Engineer\xa0; Industry: Software Development'],
  'Alumni working in fields related to AI and robotics, such as computer vision or autonomous systems, with skills in Python and TensorFlow.': ['Major: Mathmatics; Degree: M.S; Job Title: Staff Machine Learning Engineer; Industry: Motor Vehicle Manufacturing',
   'Major: Computat

In [160]:
recommendation_template = PromptTemplate("""
You are an AI academic advisor assistant tasked with providing academic pathway recommendations for advisors to use in student consultations. Generate recommendations in a specific JSON format that is analytical, evidence-based, and structured for advisor use.

------------------------------------------------------------------------------------------------
Student Profile Summary:
{context}                                
------------------------------------------------------------------------------------------------

Available Data for Recommendations:

ALUMNI PROFILES AND CAREER PATHS:
{alumni_profiles}

INDUSTRY INSIGHTS AND NOTABLE FIGURES:
{internet_insights}

------------------------------------------------------------------------------------------------

Generate a JSON response with 5 academic pathway recommendations (3 based on alumni pathways, 1 on emerging trends, 1 inspired by a notable figure).

Required JSON structure:
{{
  "recommendations": [
    {{
      "id": 1,
      "type": "alumni",  // Use "alumni", "trend", or "figure"
      "quickView": {{
        "title": "string", // Clear pathway name
        "summary": "string", // 2-3 sentences analyzing student fit
        "keyPoints": [  // Array of 3 key considerations
          "string",
          "string",
          "string"
        ],
        "nextStep": "string" // One concrete next discussion point
      }},
      "detailedView": {{
        "reasoning": "string", // Detailed analysis of fit and alignment
        "evidence": {{
          "alumniPatterns": "string", // Relevant alumni examples and patterns
          "industryContext": "string"  // Industry trends and developments
        }},
        "discussionPoints": [  // Array of specific points for advisor-student discussion
          "string",
          "string",
          "string"
        ]
      }}
    }}
  ]
}}

Requirements for content:
1. QuickView section:
   - Title: Clear and specific academic pathway name
   - Summary: Concise analysis of student fit using advisor-facing tone
   - KeyPoints: Three most important factors for consideration
   - NextStep: Specific discussion point or action for advisor

2. DetailedView section:
   - Reasoning: In-depth analysis without repeating quick view information
   - Evidence: Specific examples and data from alumni and industry sources
   - DiscussionPoints: Concrete topics for advisor-student conversations

Remember:
- Write for advisors as the primary audience ("the student" not "you/your")
- Focus on academic pathways that lead to career outcomes
- Keep suggestions broad enough for different universities
- Provide evidence-based recommendations
- Include specific discussion points for advisors
- Ensure JSON structure is valid with proper escaping of quotes

Follow these type distributions:
- Recommendations 1-3: type "alumni"
- Recommendation 4: type "trend"
- Recommendation 5: type "figure"

""")


In [161]:
recommendation_prompt = recommendation_template.format(
    context=llm_response_text,
    alumni_profiles=combined_responses["alumni_profiles"],
    internet_insights=combined_responses["internet_insights"]
)
print(recommendation_prompt)


You are an AI academic advisor assistant tasked with providing academic pathway recommendations for advisors to use in student consultations. Generate recommendations in a specific JSON format that is analytical, evidence-based, and structured for advisor use.

------------------------------------------------------------------------------------------------
Student Profile Summary:
1. CORE INTERESTS AND ABILITIES
- The student has a strong interest in AI and robotics, with a particular aptitude for mathematical modeling, programming, problem-solving, and algorithm development. These interests are supported by their favorite courses, Machine Learning and Advanced Mathematics, and their extracurricular activities, such as leading a robotics club project.
- A clear theme is the student's focus on technical and innovative fields, particularly in AI and robotics, which aligns with their career aspirations to become an AI researcher or robotics engineer.
- There is a strong alignment between

In [ ]:
final_response = llm.complete(recommendation_prompt)  
response_text = final_response.text.strip() 


In [164]:

# Remove Markdown code block markers
if response_text.startswith("```json") and response_text.endswith("```"):
    response_text = response_text[7:-3].strip()  # Strip the markers

try:
    # Attempt to parse the cleaned JSON output
    parsed_output = json.loads(response_text)
    print("JSON Output Successfully Parsed!")

    # Optionally display the JSON in a Markdown-friendly format
    display(Markdown(f"```json\n{json.dumps(parsed_output, indent=4)}\n```"))

except json.JSONDecodeError as e:
    print("Error: LLM output is not valid JSON!")
    print(f"Details: {e}")
    print("Raw Output for Debugging:")
    print(response_text)

JSON Output Successfully Parsed!


```json
{
    "recommendations": [
        {
            "id": 1,
            "type": "alumni",
            "quickView": {
                "title": "Mathematics and Machine Learning Pathway",
                "summary": "The student\u2019s strong foundation in mathematics and programming aligns well with alumni who pursued advanced degrees in mathematics and transitioned into roles such as Machine Learning Engineers in industries like automotive and software development.",
                "keyPoints": [
                    "Strong mathematical modeling skills",
                    "Interest in AI research and robotics",
                    "Alignment with alumni career paths in machine learning"
                ],
                "nextStep": "Discuss the potential benefits of pursuing a Master's in Mathematics with a focus on AI applications."
            },
            "detailedView": {
                "reasoning": "The student's interests and skills in mathematical modeling and programming are well-suited for a pathway that combines mathematics with machine learning. Alumni who have taken this path have successfully entered roles that leverage these skills in innovative industries.",
                "evidence": {
                    "alumniPatterns": "Alumni with a Mathematics M.S. have become Staff Machine Learning Engineers in motor vehicle manufacturing and Data Scientists in software development.",
                    "industryContext": "The integration of AI in industries like automotive is growing, with a demand for skills in machine learning and algorithm development."
                },
                "discussionPoints": [
                    "Explore the student's interest in pursuing a Master's degree in Mathematics.",
                    "Discuss potential career outcomes in machine learning and AI research.",
                    "Consider the student's willingness to engage in interdisciplinary projects."
                ]
            }
        },
        {
            "id": 2,
            "type": "alumni",
            "quickView": {
                "title": "Information Systems and Technology Pathway",
                "summary": "The student's technical skills and leadership in collaborative projects align with alumni who pursued Information Systems and Technology, leading to roles in data engineering and program management.",
                "keyPoints": [
                    "Leadership in collaborative environments",
                    "Technical skills in Python and TensorFlow",
                    "Interest in real-world impact projects"
                ],
                "nextStep": "Evaluate the potential for a Master's in Information Systems and Technology focusing on AI and robotics."
            },
            "detailedView": {
                "reasoning": "The student's leadership in robotics projects and technical proficiency suggest a strong fit for a pathway in Information Systems and Technology. Alumni in this field have successfully led innovative projects with significant real-world impact.",
                "evidence": {
                    "alumniPatterns": "Alumni with a degree in Information Systems and Technology have become Engineer Managers and Data Engineers in industries like entertainment and food services.",
                    "industryContext": "There is a growing need for professionals who can manage and innovate within collaborative tech environments."
                },
                "discussionPoints": [
                    "Discuss the student's interest in leadership roles within tech projects.",
                    "Explore the potential for interdisciplinary applications of AI and robotics.",
                    "Consider the student's preference for impactful, real-world projects."
                ]
            }
        },
        {
            "id": 3,
            "type": "alumni",
            "quickView": {
                "title": "Computational Mathematics and Data Science Pathway",
                "summary": "The student's proficiency in programming and algorithm development is well-suited for a pathway in Computational Mathematics, leading to data science roles in software development.",
                "keyPoints": [
                    "Proficiency in programming and algorithm development",
                    "Interest in data-driven AI applications",
                    "Alignment with data science career paths"
                ],
                "nextStep": "Consider the benefits of a Master's in Computational Mathematics with a focus on data science."
            },
            "detailedView": {
                "reasoning": "The student's skills in programming and algorithm development align with alumni who pursued Computational Mathematics and transitioned into data science roles, where these skills are crucial.",
                "evidence": {
                    "alumniPatterns": "Alumni with a Computational Mathematics M.S. have become Data Scientists in software development.",
                    "industryContext": "Data science is a rapidly growing field, with increasing demand for skills in algorithm development and data analysis."
                },
                "discussionPoints": [
                    "Explore the student's interest in data science and analytics.",
                    "Discuss potential career paths in software development and AI.",
                    "Consider the student's adaptability to interdisciplinary roles."
                ]
            }
        },
        {
            "id": 4,
            "type": "trend",
            "quickView": {
                "title": "AI and Robotics in Cybersecurity and Automation",
                "summary": "The student's interest in AI and robotics aligns with emerging trends in cybersecurity and automation, where skills in mathematical modeling and programming are increasingly valuable.",
                "keyPoints": [
                    "Emerging opportunities in AI-driven cybersecurity",
                    "Integration of robotics in automation processes",
                    "Growing demand for service robots"
                ],
                "nextStep": "Discuss the potential for specializing in AI applications within cybersecurity and automation."
            },
            "detailedView": {
                "reasoning": "The student's skills in AI and robotics are well-positioned to take advantage of emerging trends in cybersecurity and automation. These fields are experiencing significant growth, with a need for professionals who can develop and implement AI-driven solutions.",
                "evidence": {
                    "alumniPatterns": "Alumni have successfully integrated AI skills in industries like automotive and logistics, where automation is key.",
                    "industryContext": "In 2024, AI is increasingly used in cybersecurity and automation, with a rising demand for service robots due to staff shortages."
                },
                "discussionPoints": [
                    "Explore the student's interest in cybersecurity and automation.",
                    "Discuss the potential for AI applications in these fields.",
                    "Consider the student's readiness to engage with emerging technologies."
                ]
            }
        },
        {
            "id": 5,
            "type": "figure",
            "quickView": {
                "title": "Inspired by Francois Chollet: Deep Learning and Innovation",
                "summary": "The student's focus on AI and innovation aligns with the career journey of Francois Chollet, a notable figure in deep learning and AI development, emphasizing the importance of impactful technology.",
                "keyPoints": [
                    "Interest in deep learning and AI innovation",
                    "Alignment with impactful technology development",
                    "Potential for contributions to AI research"
                ],
                "nextStep": "Discuss the potential for pursuing research opportunities in deep learning and AI innovation."
            },
            "detailedView": {
                "reasoning": "The student's interests in AI and innovation resonate with the career of Francois Chollet, who has made significant contributions to deep learning and AI. This pathway emphasizes the importance of developing impactful technologies and advancing human knowledge.",
                "evidence": {
                    "alumniPatterns": "Notable figures like Francois Chollet have successfully integrated AI skills to drive innovation and address real-world challenges.",
                    "industryContext": "Deep learning continues to be a transformative force in AI, with opportunities for research and development in various sectors."
                },
                "discussionPoints": [
                    "Explore the student's interest in deep learning and AI research.",
                    "Discuss potential contributions to impactful technology development.",
                    "Consider the student's readiness to engage in cutting-edge research."
                ]
            }
        }
    ]
}
```

In [165]:
def display_recommendations(response_text):

    try:
        recommendations = json.loads(response_text)

        if "recommendations" not in recommendations:
            print("Error: 'recommendations' key not found in the JSON!")
            return
        
        print("\n🎓 **Academic Pathway Recommendations** 🎓\n")
        for rec in recommendations["recommendations"]:
            print(f"**Recommendation {rec['id']}: {rec['quickView']['title']}**")
            print(f"**Type**: {rec['type'].capitalize()}")
            print(f"**Summary**: {rec['quickView']['summary']}")
            print("\n**Key Points:**")
            for idx, point in enumerate(rec['quickView']['keyPoints'], 1):
                print(f"  {idx}. {point}")
            
            print(f"\n**Next Step:** {rec['quickView']['nextStep']}")
            print("\n**Detailed Reasoning:**")
            print(f"  {rec['detailedView']['reasoning']}")
            
            print("\n**Evidence:**")
            print(f"  - Alumni Patterns: {rec['detailedView']['evidence']['alumniPatterns']}")
            print(f"  - Industry Context: {rec['detailedView']['evidence']['industryContext']}")
            
            print("\n**Discussion Points:**")
            for idx, point in enumerate(rec['detailedView']['discussionPoints'], 1):
                print(f"  {idx}. {point}")
            
            print("\n" + "-"*80)  # Separator between recommendations

    except json.JSONDecodeError as e:
        print("Error: Unable to parse JSON!")
        print(f"Details: {e}")


In [166]:
# Clean the Markdown formatting first
if response_text.startswith("```json") and response_text.endswith("```"):
    response_text = response_text[7:-3].strip()

# Display the recommendations
display_recommendations(response_text)


🎓 **Academic Pathway Recommendations** 🎓

**Recommendation 1: Mathematics and Machine Learning Pathway**
**Type**: Alumni
**Summary**: The student’s strong foundation in mathematics and programming aligns well with alumni who pursued advanced degrees in mathematics and transitioned into roles such as Machine Learning Engineers in industries like automotive and software development.

**Key Points:**
  1. Strong mathematical modeling skills
  2. Interest in AI research and robotics
  3. Alignment with alumni career paths in machine learning

**Next Step:** Discuss the potential benefits of pursuing a Master's in Mathematics with a focus on AI applications.

**Detailed Reasoning:**
  The student's interests and skills in mathematical modeling and programming are well-suited for a pathway that combines mathematics with machine learning. Alumni who have taken this path have successfully entered roles that leverage these skills in innovative industries.

**Evidence:**
  - Alumni Patterns: A

In [153]:

llm_o1 = OpenAI(model="o1-preview-2024-09-12")
final_response = llm_o1.complete(recommendation_prompt)
final_response.text.strip()

Markdown(final_response.text.strip())

**Recommendation 1: Embrace Computational Mathematics for Advanced AI**

Given your strong aptitude for mathematical modeling and programming, diving deeper into Computational Mathematics could be a natural and rewarding academic direction for you. This field will enhance your ability to develop sophisticated algorithms and models essential for pushing the boundaries of AI and robotics.

Your favorite courses—Machine Learning and Advanced Mathematics—along with your experience leading a robotics club project and working on an autonomous drone, highlight your proficiency and passion for complex problem-solving. By focusing on Computational Mathematics, you can strengthen the theoretical foundations that underpin AI technologies, directly aligning with your aspiration to become an AI researcher or robotics engineer.

In this pathway, exploring areas such as numerical methods, optimization, and statistical modeling will be invaluable. These subjects are crucial for developing advanced machine learning algorithms and improving robotic perception and decision-making. Delving into these topics will equip you with the skills to innovate and create AI systems with significant real-world impact, a value you've consistently demonstrated.

To build experience, consider engaging in research projects or internships that focus on algorithm development or computational modeling. Participating in AI competitions can also sharpen your skills and provide practical applications for your knowledge. This approach mirrors the journey of alumni like the Data Scientist with a degree in Computational Mathematics who excels in software development, leveraging strong mathematical and programming skills to drive innovation.

As a next step, seek out opportunities to collaborate with faculty on research initiatives or explore interdisciplinary projects that combine mathematics, AI, and robotics. This will not only enhance your technical expertise but also expand your professional network, setting a strong foundation for your future career.

---

**Recommendation 2: Integrate Information Systems with AI for Innovative Solutions**

Your passion for innovation and working on projects with real-world impact suggests an academic path in Information Systems and Technology enriched with AI and robotics studies. This direction allows you to apply your technical skills within broader technological ecosystems, enhancing your ability to develop impactful solutions collaboratively.

You've demonstrated leadership and a collaborative spirit through your robotics club project and mentoring high school students. By delving into Information Systems, you can learn how to design, manage, and improve complex technological infrastructures, which is essential for bringing AI and robotics innovations to life.

Studying subjects like systems architecture, data analytics, and user-centered design will complement your existing skills. Understanding how AI systems integrate with other technologies enables you to create more effective and user-friendly applications. This aligns well with your preference for collaborative environments and projects that have tangible benefits.

Consider engaging in extracurricular activities like joining tech-focused student organizations or participating in hackathons. These experiences can enhance your teamwork and leadership skills while providing practical experience in developing solutions. Alumni such as the Engineering Manager in the entertainment industry leveraged their Information Systems background to lead impactful technology projects, reflecting the potential of this path.

To move forward, you might look for internship opportunities that allow you to work on large-scale technology projects or explore collaborative research initiatives. These experiences will not only build your resume but also deepen your understanding of how to drive innovation within complex systems.

---

**Recommendation 3: Apply AI and Robotics Skills in Interdisciplinary Fields**

Expanding your academic journey to include interdisciplinary applications of AI and robotics can open new avenues for innovation and impact. By integrating fields such as healthcare technology or environmental science, you can apply your technical expertise to solve critical real-world challenges.

Your strong background in AI, demonstrated by your internship in computer vision algorithms and autonomous drone project, positions you well to contribute to areas like biomedical engineering or environmental monitoring. Courses in these fields will broaden your perspective and reveal how AI and robotics can be leveraged beyond traditional industries.

Engaging in interdisciplinary research projects or collaborative teams can enhance your adaptability and problem-solving skills. Your experience mentoring others and leading projects shows you thrive in collaborative settings. Alumni like the Data Engineer working in health and wellness utilized their skills in Information Systems and AI to make significant contributions to healthcare technology.

To gain practical experience, consider internships or volunteer opportunities with organizations focused on healthcare innovation or environmental sustainability. This hands-on involvement will deepen your understanding of how AI can drive progress in these vital sectors.

As immediate steps, explore introductory courses or seminars in areas that intersect with AI, such as bioinformatics or environmental data analysis. Connecting with faculty who specialize in interdisciplinary research can also provide guidance and potential mentorship opportunities.

---

**Recommendation 4: Focus on Emerging Trends in AI and Robotics**

Aligning your academic path with emerging industry trends can position you at the cutting edge of AI and robotics. Concentrating on areas like AI in cybersecurity, mobile robotics, or the integration of digital twins offers exciting opportunities to innovate and lead in rapidly evolving fields.

Your solid foundation in machine learning and programming is ideal for delving into these advanced topics. Studying subjects such as advanced machine learning applications, robotic automation, and cyber-physical systems will keep you at the forefront of technological advancements. This focus resonates with your desire to engage in projects with real-world impact and to drive innovation.

Participating in research focused on emerging technologies or attending specialized workshops can enhance your knowledge and skills. The industry's growing interest in areas like AI-driven cybersecurity and intelligent automation means that expertise in these domains is highly valued. Engaging with these trends ensures your skills remain relevant and sought after.

To build experience, seek internships or collaborative projects with companies or research labs pioneering these technologies. This hands-on experience not only bolsters your resume but also provides insights into the practical applications of your studies.

Begin by identifying faculty members or industry professionals working on these cutting-edge topics and reach out to learn more about their work. Staying informed about industry developments through journals, conferences, and professional networks will also help you identify opportunities and stay ahead of the curve.

---

**Recommendation 5: Follow in the Footsteps of AI Innovators**

Drawing inspiration from notable figures like François Chollet, who created Keras, you might consider an academic path that emphasizes both theoretical depth and practical innovation in AI. Embracing a journey similar to these innovators involves deeply engaging with AI research while contributing to the development of tools and technologies that advance the field.

Your proficiency in Python and TensorFlow and your enthusiasm for algorithm development align well with the paths of these leaders. Focusing on advanced AI topics such as deep learning architectures, neural networks, and artificial general intelligence can position you to make significant contributions to AI research.

Engaging in open-source projects or developing your own AI tools can provide practical experience and showcase your initiative. François Chollet's success illustrates how creating accessible AI frameworks can have a profound impact on the industry. Emulating this approach allows you to blend your technical skills with your value of innovation.

Collaborating with researchers, publishing papers, or presenting at conferences can further enhance your academic journey. Alumni who pursued advanced degrees, like the Staff Machine Learning Engineer with a Ph.D. in Mathematics, have reached influential positions in industries like motor vehicle manufacturing by combining deep research with practical applications.

To take immediate action, consider reaching out to faculty involved in cutting-edge AI research to inquire about assistantship opportunities. Participating in AI-focused seminars or contributing to scholarly publications can also set you on a path toward meaningful innovation and leadership in the field.

---

These recommendations are designed to align with your interests, strengths, and values, offering various pathways to achieve your career aspirations. Each path provides opportunities to deepen your expertise, engage in impactful projects, and build a foundation for a successful future in AI and robotics.

In [151]:
final_response = llm.complete(recommendation_prompt)
final_response.text.strip()

Markdown(final_response.text.strip())

**Recommendation 1: Pursuing a Path in Computational Mathematics and AI Research**

Given your strong interest in AI and robotics, coupled with your aptitude for mathematical modeling and programming, a pathway in computational mathematics could be a perfect fit. This direction aligns seamlessly with your career aspirations to become an AI researcher, as it emphasizes the development of sophisticated algorithms and models that are foundational to AI systems.

Your background in machine learning and advanced mathematics, along with your experience in computer vision algorithms, positions you well for this path. Computational mathematics will deepen your understanding of the mathematical underpinnings of AI, enhancing your ability to innovate in this field. This aligns with your values of technical excellence and real-world impact, as seen in your autonomous drone project and leadership in the robotics club.

Academically, exploring areas such as numerical analysis, optimization, and data science will be crucial. These areas are integral to developing the skills needed for AI research, as they focus on creating efficient algorithms and models. This flexibility allows you to apply these skills across various AI applications, from robotics to data-driven decision-making.

To build relevant experience, consider participating in AI research projects or competitions. Engaging in collaborative research environments will hone your problem-solving skills and expose you to cutting-edge technology. Alumni like those who became Staff Machine Learning Engineers in the motor vehicle manufacturing industry have followed similar paths, leveraging their computational skills to drive innovation.

As a practical next step, seek out research assistant positions or internships in AI labs. These opportunities will provide hands-on experience and mentorship, helping you refine your skills and expand your professional network. This approach will set a strong foundation for a successful career in AI research.

**Recommendation 2: Exploring Interdisciplinary Roles in AI and Robotics**

Your technical skills and innovative mindset make you an excellent candidate for interdisciplinary roles that apply AI and robotics in diverse fields. This pathway allows you to leverage your expertise in Python and TensorFlow while exploring new applications, such as healthcare technology or environmental solutions.

Your experience in mentoring and leading projects demonstrates your ability to work collaboratively, a key asset in interdisciplinary roles. By integrating AI with other disciplines, you can contribute to impactful projects that address real-world challenges, aligning with your values of innovation and collaboration.

Academically, consider exploring areas like bioinformatics, environmental engineering, or cognitive science. These fields offer opportunities to apply AI and robotics in novel ways, broadening your perspective and enhancing your adaptability. This approach aligns with alumni who have successfully transitioned into roles like Clinical Data Engineer in the health and wellness industry.

To build experience, engage in interdisciplinary projects or internships that combine AI with other fields. Participating in hackathons or innovation challenges can also provide exposure to diverse applications and foster creative problem-solving. These experiences will help you develop a versatile skill set, preparing you for a dynamic career.

As a next step, reach out to professionals in interdisciplinary roles to learn about their experiences and seek mentorship. This will provide valuable insights and help you identify potential career paths that align with your interests and values.

**Recommendation 3: Advancing in Robotics Engineering with a Focus on Autonomous Systems**

Your passion for robotics and experience with autonomous drones suggest a promising pathway in robotics engineering, particularly in developing autonomous systems. This field is rapidly evolving, with significant opportunities to innovate and create impactful solutions.

Your skills in algorithm development and problem-solving are essential for designing and optimizing autonomous systems. This aligns with your career goal of becoming a robotics engineer, as it involves creating intelligent systems that can operate independently in complex environments.

Academically, focus on areas such as control systems, machine learning, and embedded systems. These areas are critical for understanding the mechanics and intelligence behind autonomous robots, enabling you to design systems that can navigate and interact with the world autonomously.

To gain practical experience, participate in robotics competitions or join research projects focused on autonomous systems. These activities will enhance your technical skills and provide opportunities to collaborate with peers and industry professionals. Alumni who have become Senior System Engineers in the defense and space industry have followed similar paths, applying their expertise to develop cutting-edge technology.

As a practical next step, consider applying for internships with companies or research institutions specializing in autonomous systems. These experiences will provide hands-on learning and help you build a network of contacts in the robotics field, paving the way for a successful career.

**Recommendation 4: Embracing Emerging Trends in AI and Robotics**

The latest industry trends highlight exciting opportunities in AI and robotics, particularly in areas like cybersecurity and mobile manipulators. Embracing these trends can position you at the forefront of innovation, aligning with your interest in cutting-edge technology and real-world impact.

Your skills in mathematical modeling and programming are crucial for developing AI systems that address complex challenges, such as enhancing cybersecurity or optimizing robotic automation. This aligns with your values of technical excellence and innovation, as seen in your extracurricular activities and career aspirations.

Academically, explore areas like cybersecurity, robotics automation, and digital twins. These fields are at the intersection of AI and robotics, offering opportunities to apply your skills in innovative ways. This approach aligns with industry trends, where the demand for service robots and intelligent automation solutions is rapidly growing.

To build experience, consider participating in industry conferences or workshops focused on emerging AI and robotics trends. Engaging with professionals and thought leaders in these areas will provide valuable insights and help you stay informed about the latest developments.

As a next step, seek out projects or internships that focus on emerging technologies in AI and robotics. These experiences will allow you to apply your skills in real-world settings, preparing you for a dynamic and impactful career.

**Recommendation 5: Following the Academic Journey of Notable AI Figures**

Drawing inspiration from notable AI researchers like Francois Chollet, who has made significant contributions to deep learning and AI, you can chart a path that emphasizes innovation and real-world impact. Chollet's work in developing Keras and advancing AI technology aligns with your interests and values, providing a model for your academic journey.

Your proficiency in Python and TensorFlow, along with your experience in AI competitions, positions you well to follow a similar path. By focusing on deep learning and AI research, you can contribute to advancements in AI technology, aligning with your career goals and desire to work on impactful projects.

Academically, delve into areas like deep learning, neural networks, and artificial general intelligence. These fields are central to AI research and offer opportunities to explore the potential of AI in advancing human knowledge. This approach aligns with the work of influential figures like Chollet, who have driven innovation in AI.

To build experience, engage in research projects or internships focused on deep learning and AI development. Collaborating with researchers and participating in AI conferences will provide valuable insights and help you build a network of contacts in the field.

As a practical next step, consider applying for research assistant positions in AI labs or pursuing graduate studies in AI. These opportunities will provide the foundation for a successful career in AI research, allowing you to contribute to the advancement of AI technology.

In [147]:
final_response = llm.complete(recommendation_prompt)
final_response.text.strip()

Markdown(final_response.text.strip())

RECOMMENDATION 1:
Title: Advanced Computational Mathematics and AI
Key Academic Choices:
- Major/Minor Combination: Major in Computational Mathematics with a Minor in Artificial Intelligence
- Essential Courses: Advanced Calculus, Linear Algebra, Machine Learning, Deep Learning, Computer Vision, and Algorithm Design
- Complementary Activities: Join the AI and Robotics Club, participate in AI competitions, and engage in research projects focused on AI applications
Pathway Benefits: This pathway strengthens the student's mathematical modeling and programming skills, essential for AI research and robotics engineering roles.
Alumni/Industry Evidence: Alumni with similar backgrounds have become Staff Machine Learning Engineers and Data Scientists in industries like software development and motor vehicle manufacturing.
Next Steps: Meet with an academic advisor to plan course sequences and explore research opportunities in AI labs.
Additional Considerations: Ensure prerequisites for advanced courses are met early in the academic plan.

RECOMMENDATION 2:
Title: Robotics Engineering and Systems Design
Key Academic Choices:
- Major/Minor Combination: Major in Robotics Engineering with a Minor in Systems Design
- Essential Courses: Robotics Systems, Control Systems, Embedded Systems, Autonomous Systems, and Systems Engineering
- Complementary Activities: Lead projects in the Robotics Club, participate in hackathons, and intern with companies focusing on autonomous systems
Pathway Benefits: This pathway provides hands-on experience in designing and implementing robotic systems, aligning with the student's goal to become a robotics engineer.
Alumni/Industry Evidence: Alumni have pursued careers as Senior System Engineers and Site Reliability Engineers in defense and space manufacturing.
Next Steps: Explore internship opportunities in robotics companies and seek mentorship from faculty in the robotics department.
Additional Considerations: Balance theoretical courses with practical labs to gain comprehensive skills.

RECOMMENDATION 3:
Title: Data Science and AI Applications
Key Academic Choices:
- Major/Minor Combination: Major in Data Science with a Concentration in AI Applications
- Essential Courses: Data Structures, Statistical Learning, Natural Language Processing, AI Ethics, and Big Data Analytics
- Complementary Activities: Participate in data science competitions, join the Data Science Society, and collaborate on interdisciplinary projects
Pathway Benefits: This pathway equips the student with data analysis and AI application skills, preparing them for roles in AI research and data engineering.
Alumni/Industry Evidence: Alumni have become Data Engineers and Analytics Engineers in industries like entertainment and food and beverage manufacturing.
Next Steps: Enroll in introductory data science courses and attend workshops on AI tools like TensorFlow.
Additional Considerations: Consider summer courses to accelerate learning in key areas.

RECOMMENDATION 4:
Title: Emerging Field Preparation in AI and Cybersecurity
Key Academic Choices:
- Emerging Field: AI in Cybersecurity
- Essential Courses: Cybersecurity Fundamentals, AI in Cybersecurity, Network Security, and Ethical Hacking
- Complementary Activities: Join cybersecurity clubs, participate in capture-the-flag competitions, and work on projects integrating AI with cybersecurity solutions
Pathway Benefits: Prepares the student for emerging opportunities in AI-driven cybersecurity, a growing field due to increasing cyber threats.
Alumni/Industry Evidence: Industry trends indicate a rising demand for AI skills in cybersecurity, with significant career opportunities.
Next Steps: Research academic programs offering specialized courses in AI and cybersecurity and attend industry conferences.
Additional Considerations: Stay updated on the latest cybersecurity threats and AI solutions.

RECOMMENDATION 5:
Title: Inspirational Academic Journey of John J. Leonard
Key Academic Choices:
- Highlight: John J. Leonard's focus on robotics and SLAM technology
- Similar Academic Paths: Pursue a major in Robotics Engineering with courses in SLAM, Autonomous Navigation, and Machine Learning
- Complementary Activities: Engage in research projects on autonomous mobile robots and participate in national robotics competitions
Pathway Benefits: Emulating Leonard's journey can lead to innovative contributions in robotics and AI, with real-world applications.
Alumni/Industry Evidence: Leonard's work in robotics has led to significant advancements and recognition in the field.
Next Steps: Connect with faculty involved in robotics research and seek opportunities to contribute to ongoing projects.
Additional Considerations: Consider graduate studies to deepen expertise in robotics and AI.

In [142]:
final_response = llm.complete(recommendation_prompt)
final_response.text.strip()

Markdown(final_response.text.strip())


RECOMMENDATION 1:
Title: Staff Machine Learning Engineer in Motor Vehicle Manufacturing
Description: This career path involves applying AI and machine learning to enhance vehicle manufacturing processes, focusing on automation and efficiency.
Why It Fits: The student's interest in AI and robotics aligns with the technical demands of this role, which requires skills in mathematical modeling and programming.
Action Steps: 
1. Pursue advanced coursework or certifications in machine learning and AI applications in manufacturing.
2. Seek internships or projects related to AI in the automotive industry.
3. Network with professionals in the motor vehicle manufacturing sector to gain insights and mentorship.
Additional Insights: This role offers the opportunity to work on cutting-edge technology that impacts real-world manufacturing processes.

RECOMMENDATION 2:
Title: Data Scientist in Software Development
Description: This role involves using data analysis and machine learning to develop software solutions, focusing on innovation and problem-solving.
Why It Fits: The student's skills in Python, TensorFlow, and algorithm development are directly applicable to this career path.
Action Steps:
1. Engage in software development projects that incorporate AI and machine learning.
2. Participate in hackathons or coding competitions to enhance problem-solving skills.
3. Build a portfolio showcasing projects that demonstrate expertise in data science and software development.
Additional Insights: This path allows for creativity and innovation in developing software that leverages AI technologies.

RECOMMENDATION 3:
Title: Clinical Data Engineer in Health, Wellness & Fitness
Description: This interdisciplinary role applies AI and data engineering skills to improve healthcare technology and patient outcomes.
Why It Fits: The student's technical skills can be applied to impactful projects in healthcare, aligning with their values of innovation and real-world impact.
Action Steps:
1. Explore coursework or certifications in healthcare technology and data engineering.
2. Volunteer or intern with organizations focused on health tech innovations.
3. Attend conferences or workshops on AI applications in healthcare to stay informed about industry trends.
Additional Insights: This role offers the chance to make a significant impact on healthcare systems and patient care through technology.

RECOMMENDATION 4:
Title: Mobile Manipulators in Robotics Engineering
Description: The rise of mobile manipulators presents an opportunity to work on advanced robotics systems that integrate AI for tasks in industries like automotive and logistics.
Why It Fits: The student's interest in robotics and AI aligns with the technical challenges and innovations in developing mobile manipulators.
Action Steps:
1. Research and study the latest advancements in mobile manipulators and robotics.
2. Participate in robotics competitions or projects that focus on mobile manipulator technology.
3. Connect with industry professionals working on mobile robotics to gain insights and potential collaboration opportunities.
Additional Insights: This trend represents a growing field with significant potential for innovation and impact across various industries.

RECOMMENDATION 5:
Title: John J. Leonard - Roboticist and Professor at MIT
Description: John J. Leonard's career in robotics, particularly in SLAM for autonomous robots, exemplifies innovation and real-world impact.
Why It Fits: Leonard's work in robotics and AI aligns with the student's aspirations to become an AI researcher or robotics engineer.
Action Steps:
1. Study Leonard's research and publications to understand his approach to robotics and AI.
2. Engage in projects or research that focus on SLAM and autonomous systems.
3. Consider pursuing advanced studies or research opportunities at institutions known for robotics innovation, such as MIT.
Additional Insights: Leonard's career demonstrates the importance of combining technical expertise with innovative thinking to address complex challenges in robotics.